In [1]:
!pip install medmnist
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=f2aab55fc1026af8ae6c73066d5763d6df5887259950734b66e836300d84f0a3
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torch.utils.data import random_split
from medmnist import OCTMNIST, PneumoniaMNIST, RetinaMNIST,BreastMNIST

# Define the transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean for RGB
                         [0.229, 0.224, 0.225])  # ImageNet std for RGB
])

# Custom Dataset class to handle merged datasets and apply offsets
class CustomDataset(Dataset):
    def __init__(self, datasets, offsets, transform=None):
        self.datasets = datasets
        self.offsets = offsets
        self.transform = transform
        self.merged_samples = self._merge_datasets()

    def _merge_datasets(self):
        samples = []
        for name, dataset in self.datasets.items():
            offset = self.offsets[name]
            samples.extend(self.offset_dataset(dataset, offset))
        return samples

    def offset_dataset(self, dataset, offset):
        new_samples = []
        for x, y in dataset:
            new_y = torch.tensor([y[0] + offset])
            new_samples.append((x, new_y))
        return new_samples

    def __len__(self):
        return len(self.merged_samples)

    def __getitem__(self, idx):
        x, y = self.merged_samples[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

# Load the datasets
datasets = {
    'oct': OCTMNIST(split='train', transform=None, download=True),
    'pneu': PneumoniaMNIST(split='train', transform=None, download=True),
    'retina': RetinaMNIST(split='train', transform=None, download=True),
    'breast': BreastMNIST(split='train', transform=None, download=True)
}

# Set offsets to distinguish different classes in each dataset
offsets = {'oct': 0, 'pneu': 4, 'retina': 6, 'breast': 11}

# Create the custom dataset and apply transformations
custom_dataset = CustomDataset(datasets, offsets, transform=transform)

len1=len(custom_dataset)//2
len2=len(custom_dataset) - len1

first_half, second_half = random_split(custom_dataset, [len1, len2])
loader1 = DataLoader(first_half, batch_size=32, shuffle=True)
loader2 = DataLoader(second_half, batch_size=32, shuffle=True)

print(f"Train_loader1 size:{len(loader1)}")
print(f"Train_loader2 size:{len(loader2)}")
#Creating the val datasets 
test_datasets = {
    'oct': OCTMNIST(split='test', transform=None, download=True),
    'pneu': PneumoniaMNIST(split='test', transform=None, download=True),
    'retina': RetinaMNIST(split='test', transform=None, download=True),
    'breast': BreastMNIST(split='test', transform=None, download=True)
}

offsets = {'oct': 0, 'pneu': 4, 'retina': 6, 'breast': 11}

# Create evaluation dataset and loader
eval_dataset = CustomDataset(test_datasets, offsets, transform=transform)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)
print(f"Eval_loader size:{len(eval_loader)}")

100%|██████████| 54.9M/54.9M [00:07<00:00, 6.87MB/s]


100%|██████████| 4.17M/4.17M [00:00<00:00, 7.30MB/s]


100%|██████████| 3.29M/3.29M [00:00<00:00, 6.99MB/s]


100%|██████████| 560k/560k [00:00<00:00, 4.50MB/s]


Train_loader1 size:1623
Train_loader2 size:1623
Using downloaded and verified file: /root/.medmnist/octmnist.npz
Using downloaded and verified file: /root/.medmnist/pneumoniamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz
Using downloaded and verified file: /root/.medmnist/breastmnist.npz
Eval_loader size:69


In [3]:
import torch
import torch.nn as nn
from torchvision import models
import os

# Enable synchronous CUDA errors
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Load pretrained VGG16 model
model = models.vgg16(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False
model.classifier[6] = nn.Linear(4096, 13)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# weights_path = '/kaggle/working/vgg16_model.pth'  # Adjust path if needed in Kaggle
# state_dict = torch.load(weights_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
# model.load_state_dict(state_dict)

# 3. Move model to appropriate device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training Loop
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(loader1):  # Replace 'loader' with your DataLoader
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.squeeze(dim=1)

        # Validate labels
        if labels.max() >= 13 or labels.min() < 0:
            print(f"Batch {i} labels: {labels}")
            raise ValueError("Labels must be in range [0, 12]")

        # Ensure correct dtype
        labels = labels.long()  # Force to torch.long if not already

        optimizer.zero_grad()
        outputs = model(inputs)

        # Check for NaN/Inf (optional debugging)
        if torch.isnan(outputs).any() or torch.isinf(outputs).any():
            print(f"Batch {i} outputs contain NaN/Inf: {outputs}")
            raise ValueError("Invalid outputs")
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(loader1):.4f}")
    torch.cuda.empty_cache()

torch.save(model.state_dict(), 'vgg16_model.pth')
print("Model saved successfully.")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 233MB/s]


Epoch [1/10], Loss: 0.5102
Epoch [2/10], Loss: 0.3933
Epoch [3/10], Loss: 0.3399
Epoch [4/10], Loss: 0.2980
Epoch [5/10], Loss: 0.2647
Epoch [6/10], Loss: 0.2333
Epoch [7/10], Loss: 0.2074
Epoch [8/10], Loss: 0.1818
Epoch [9/10], Loss: 0.1625
Epoch [10/10], Loss: 0.1457
Model saved successfully.


In [4]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(loader2):  # Replace 'loader' with your DataLoader
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.squeeze(dim=1)

        # Validate labels
        if labels.max() >= 13 or labels.min() < 0:
            print(f"Batch {i} labels: {labels}")
            raise ValueError("Labels must be in range [0, 12]")

        # Ensure correct dtype
        labels = labels.long()  # Force to torch.long if not already

        optimizer.zero_grad()
        outputs = model(inputs)

        # Check for NaN/Inf (optional debugging)
        if torch.isnan(outputs).any() or torch.isinf(outputs).any():
            print(f"Batch {i} outputs contain NaN/Inf: {outputs}")
            raise ValueError("Invalid outputs")
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(loader2):.4f}")
    torch.cuda.empty_cache()

torch.save(model.state_dict(), 'vgg16_model.pth')
print("Model saved successfully.")

Epoch [1/10], Loss: 0.3988
Epoch [2/10], Loss: 0.3263
Epoch [3/10], Loss: 0.2800
Epoch [4/10], Loss: 0.2432
Epoch [5/10], Loss: 0.2086
Epoch [6/10], Loss: 0.1824
Epoch [7/10], Loss: 0.1569
Epoch [8/10], Loss: 0.1362
Epoch [9/10], Loss: 0.1196
Epoch [10/10], Loss: 0.1094
Model saved successfully.


In [5]:
# import shutil

# shutil.make_archive('zipped_file_name', 'zip', '/kaggle/working/zipped_file_name')

In [6]:
import torch
import torch.nn as nn
from torchvision import models
import os

# Enable synchronous CUDA errors
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Load pretrained VGG16 model
model = models.vgg16(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False
model.classifier[6] = nn.Linear(4096, 13)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
weights_path = '/kaggle/working/vgg16_model.pth'  # Adjust path if needed in Kaggle
state_dict = torch.load(weights_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model.load_state_dict(state_dict)

# 3. Move model to appropriate device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

<ipython-input-6-3c4f2b6e25e5>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_location=torch.device('cuda' if torch.cuda.is_avai

In [7]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

def evaluate_model(model, dataloader, device, num_classes=13):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device).squeeze()  # Squeeze targets
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)  # Probabilities for AUC
            _, predicted = torch.max(outputs, dim=1)  # Predicted classes

            # Collect predictions, labels, and probabilities
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # Accuracy
    accuracy = accuracy_score(all_labels, all_preds)

    # F1 Score (macro average for multi-class)
    f1 = f1_score(all_labels, all_preds, average='macro')

    # Precision and Recall (macro average)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')

    # AUC (one-vs-rest for multi-class)
    # Convert labels to one-hot encoding for AUC calculation
    try:
        auc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='macro')
    except ValueError as e:
        print(f"AUC calculation failed: {e}")
        auc = None

    # Confusion Matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Print results
    print(f"Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")
    print(f"Precision (Macro): {precision:.4f}")
    print(f"Recall (Macro): {recall:.4f}")
    print(f"AUC (One-vs-Rest, Macro): {auc:.4f}" if auc is not None else "AUC: N/A")
    print(f"Confusion Matrix:\n{conf_matrix}")

    # Return all metrics as a dictionary
    metrics = {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'auc': auc,
        'confusion_matrix': conf_matrix
    }
    return metrics

# Example usage
metrics = evaluate_model(model, eval_loader, device)

Evaluation Metrics:
Accuracy: 0.7069
F1 Score (Macro): 0.5443
Precision (Macro): 0.6290
Recall (Macro): 0.5725
AUC (One-vs-Rest, Macro): 0.9544
Confusion Matrix:
[[234   7   1   8   0   0   0   0   0   0   0   0   0]
 [ 46 178   1  25   0   0   0   0   0   0   0   0   0]
 [115  10  22 103   0   0   0   0   0   0   0   0   0]
 [  8   1   2 239   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0 168  65   0   0   0   0   0   0   0]
 [  0   0   0   0   3 387   0   0   0   0   0   0   0]
 [  0   1   0   0   0   0 152   2   5  11   3   0   0]
 [  0   0   0   0   0   0  39   1   3   1   2   0   0]
 [  0   0   0   0   0   0  59   0   6  15  12   0   0]
 [  0   0   0   0   0   0  31   1   2  21  13   0   0]
 [  0   0   0   0   0   0  12   0   0   1   7   0   0]
 [  2   0   0   0   0   0   0   0   0   0   0  21  19]
 [  0   2   0   1   0   0   0   0   0   0   0   6 105]]
